# Import Libraries

In [35]:
import os
import re
import json
import glob
import string
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import time
from datetime import timedelta

# spacy
# import spacy
# from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [41]:
Source = '/Users/wenlanzhang/Downloads/PhD_UCL/Data/Twitter/Nairobi_1922/Sum/Summary.csv'
# used uncleaned file

Export = '/Users/wenlanzhang/Downloads/PhD_UCL/Data/Twitter/Nairobi_1922/Language/'

# Remove Websites

In [3]:
df = pd.read_csv(Source, lineterminator='\n')
df

,id,text,userid,lat,lon,created_at
0,1079755309281890305,Quick Pose Before The Vibe @KIZAnairobi @ KIZA...,149813541,-1.290832,36.782992,2018-12-31 15:05:01+00:00
1,1079759392138907649,365/365 \n#whereveryouare #haveagoodone #beTha...,208112568,-1.283333,36.816667,2018-12-31 15:21:14+00:00
2,1079760884526972928,"Happy 2019 from us @ Nairobi, Kenya https://t....",51949523,-1.283330,36.816700,2018-12-31 15:27:10+00:00
3,1079762365644660736,"We ride together, We shine together. @ Nairobi...",89940972,-1.283330,36.816700,2018-12-31 15:33:03+00:00
4,1079764407675015170,Nairobi: 6:41pm: sunset,111780330,-1.270000,36.800000,2018-12-31 15:41:10+00:00
...,...,...,...,...,...,...
335453,1542517996685369344,find only Kenyan tea and not a blend. I have o...,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00
335454,1542517994080702464,✦ [ Sakaja ]\n\nQuestion from an American\n\nh...,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00
335455,1542518238566703108,Just posted a photo @ Adams Arcade - Ngong Roa...,1339887369495916545,-1.300567,36.780725,2022-06-30 14:39:41+00:00
335456,1542521919118770184,You better recognize....\nOwner: @kemeigabriel...,839394178824028160,-1.283330,36.816700,2022-06-30 14:54:18+00:00


In [4]:
Text = df.text.tolist()
Text

['Quick Pose Before The Vibe @KIZAnairobi @ KIZA Restaurant and Lounge Nairobi https://t.co/A8ceWQ8WZY',
 '365/365 \n#whereveryouare #haveagoodone #beThankful #Godsfavor @ Nairobi https://t.co/eYbyvneLCG',
 'Happy 2019 from us @ Nairobi, Kenya https://t.co/TI1pJjXTyq',
 'We ride together, We shine together. @ Nairobi, Kenya https://t.co/HXkur0JReH',
 'Nairobi: 6:41pm: sunset',
 'We set to usher in the new year 2019 in style @villarosakempin ....Happy new year my lovely people @ Villa Rosa Kem… https://t.co/ExJeKml7NB',
 'broken clouds -&gt; scattered clouds\ntemperature up 18°C -&gt; 23°C\nhumidity down 88% -&gt; 53%\nwind 4kmh -&gt; 7kmh',
 'current weather in Nairobi: scattered clouds, 23°C\n53% humidity, wind 7kmh, pressure 1021mb',
 'Just posted a photo @ Nairobi, Kenya https://t.co/4izaNA9SWB',
 'tonight we are dundaing at churchill show pale garden city#ChurchillShowNYE https://t.co/r01qA79O1d',
 'Be true to who you are! Dont sit around waiting for approval,life is too short....D

In [7]:
import re
import string
from nltk.corpus import stopwords

def remove_stops(text, stops): 
    text = re.sub(r'Just posted a photo', '', str(text), flags=re.MULTILINE)  # Auto-generated for pic
    text = re.sub(r'Just posted a video', '', str(text), flags=re.MULTILINE)  # Auto-generated for video    
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) # URL
    text = re.sub(r'(nairobi|Nairobi)', '', text, flags=re.MULTILINE) # Nariobi
    text = re.sub(r'kenya|Kenya', '', text, flags=re.MULTILINE)  # Kenya

    words = text.split()
    final = []
    for word in words:
        if word not in stops:
            final.append(word)
    final = " ".join(final)
    final = final.translate(str.maketrans("", "", string.punctuation))
    final = "".join([i for i in final if not i.isdigit()])
    while "  " in final:
        final = final.replace("  ", " ")
    return (final)

def clean_docs(docs):
    stops = []
#     stops = stopwords.words("english")

    final = []
    for doc in docs:
        clean_doc = remove_stops(doc, stops)
        final.append(clean_doc)
    return (final)

In [8]:
cleaned_docs = clean_docs(Text)
len(cleaned_docs)
cleaned_docs

['Quick Pose Before The Vibe KIZA KIZA Restaurant and Lounge',
 ' whereveryouare haveagoodone beThankful Godsfavor ',
 'Happy from us ',
 'We ride together We shine together ',
 ' pm sunset',
 'We set to usher in the new year in style villarosakempin Happy new year my lovely people Villa Rosa Kem…',
 'broken clouds gt scattered clouds temperature up °C gt °C humidity down gt wind kmh gt kmh',
 'current weather in scattered clouds °C humidity wind kmh pressure mb',
 ' ',
 'tonight we are dundaing at churchill show pale garden cityChurchillShowNYE',
 'Be true to who you are Dont sit around waiting for approvallife is too shortDO what makes you happy',
 ' ',
 'After many years I’ll say goodbye to while in I can’t wait for I have faith that it will be full…',
 'The year that was National Museums of',
 '年越しパーティー！まだケニアは年！ New year party It is still in Happy new year 年越し newyear newyearparty …',
 '🎼😄👨\u200d🚀👁 Spray your block Stay Woke 👨\u200d💻👨\u200d🚀🤞👁🤙 streetart street streetphotography sp

In [9]:
df.text = cleaned_docs
df

,id,text,userid,lat,lon,created_at
0,1079755309281890305,Quick Pose Before The Vibe KIZA KIZA Restauran...,149813541,-1.290832,36.782992,2018-12-31 15:05:01+00:00
1,1079759392138907649,whereveryouare haveagoodone beThankful Godsfa...,208112568,-1.283333,36.816667,2018-12-31 15:21:14+00:00
2,1079760884526972928,Happy from us,51949523,-1.283330,36.816700,2018-12-31 15:27:10+00:00
3,1079762365644660736,We ride together We shine together,89940972,-1.283330,36.816700,2018-12-31 15:33:03+00:00
4,1079764407675015170,pm sunset,111780330,-1.270000,36.800000,2018-12-31 15:41:10+00:00
...,...,...,...,...,...,...
335453,1542517996685369344,find only n tea and not a blend I have ordered...,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00
335454,1542517994080702464,✦ Sakaja Question from an American,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00
335455,1542518238566703108,Adams Arcade Ngong Road,1339887369495916545,-1.300567,36.780725,2022-06-30 14:39:41+00:00
335456,1542521919118770184,You better recognize Owner kemeigabriel forest...,839394178824028160,-1.283330,36.816700,2022-06-30 14:54:18+00:00


# Language Detection

In [10]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

import time
from datetime import timedelta

def create_lang_detector(nlp, name):
    return LanguageDetector()

In [11]:
Language.factory("language_detector", func = create_lang_detector)
nlp = spacy.load('en_core_web_sm') 
nlp.add_pipe('language_detector', last = True) 

In [12]:
# Time the process

start_time = time.monotonic()

df['Language'] = df['text'].apply(lambda x: nlp(x)._.language['language']) 
df['LangAcc'] = df['text'].apply(lambda x: nlp(x)._.language['score']) 

end_time = time.monotonic()
print(timedelta(seconds = end_time - start_time))

df

# All: 1.5h

1:26:38.621665


,id,text,userid,lat,lon,created_at,Language,LangAcc
0,1079755309281890305,Quick Pose Before The Vibe KIZA KIZA Restauran...,149813541,-1.290832,36.782992,2018-12-31 15:05:01+00:00,en,0.999995
1,1079759392138907649,whereveryouare haveagoodone beThankful Godsfa...,208112568,-1.283333,36.816667,2018-12-31 15:21:14+00:00,en,0.999996
2,1079760884526972928,Happy from us,51949523,-1.283330,36.816700,2018-12-31 15:27:10+00:00,en,0.999998
3,1079762365644660736,We ride together We shine together,89940972,-1.283330,36.816700,2018-12-31 15:33:03+00:00,en,0.999997
4,1079764407675015170,pm sunset,111780330,-1.270000,36.800000,2018-12-31 15:41:10+00:00,no,0.857138
...,...,...,...,...,...,...,...,...
335453,1542517996685369344,find only n tea and not a blend I have ordered...,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00,en,0.999996
335454,1542517994080702464,✦ Sakaja Question from an American,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00,en,0.999997
335455,1542518238566703108,Adams Arcade Ngong Road,1339887369495916545,-1.300567,36.780725,2022-06-30 14:39:41+00:00,en,0.714283
335456,1542521919118770184,You better recognize Owner kemeigabriel forest...,839394178824028160,-1.283330,36.816700,2022-06-30 14:54:18+00:00,da,0.714281


In [15]:
# Check All Languages

UniqueLang = set(df.Language.tolist())
UniqueLang

{'UNKNOWN',
 'af',
 'ar',
 'bg',
 'ca',
 'cs',
 'cy',
 'da',
 'de',
 'el',
 'en',
 'es',
 'et',
 'fa',
 'fi',
 'fr',
 'gu',
 'hi',
 'hr',
 'hu',
 'id',
 'it',
 'ja',
 'kn',
 'ko',
 'lt',
 'lv',
 'mk',
 'nl',
 'no',
 'pa',
 'pl',
 'pt',
 'ro',
 'ru',
 'sk',
 'sl',
 'so',
 'sq',
 'sv',
 'sw',
 'te',
 'th',
 'tl',
 'tr',
 'uk',
 'vi',
 'zh-cn'}

In [16]:
# Non-duplicated Language Counts

len(UniqueLang)

48

In [17]:
# Export the detected output
df.to_csv(Export + 'AllLanguage.csv', index = False)

# Get High (50%) Accuracy Results

In [18]:
dfA = df.loc[df['LangAcc'] >= 0.5]
dfA

,id,text,userid,lat,lon,created_at,Language,LangAcc
0,1079755309281890305,Quick Pose Before The Vibe KIZA KIZA Restauran...,149813541,-1.290832,36.782992,2018-12-31 15:05:01+00:00,en,0.999995
1,1079759392138907649,whereveryouare haveagoodone beThankful Godsfa...,208112568,-1.283333,36.816667,2018-12-31 15:21:14+00:00,en,0.999996
2,1079760884526972928,Happy from us,51949523,-1.283330,36.816700,2018-12-31 15:27:10+00:00,en,0.999998
3,1079762365644660736,We ride together We shine together,89940972,-1.283330,36.816700,2018-12-31 15:33:03+00:00,en,0.999997
4,1079764407675015170,pm sunset,111780330,-1.270000,36.800000,2018-12-31 15:41:10+00:00,no,0.857138
...,...,...,...,...,...,...,...,...
335453,1542517996685369344,find only n tea and not a blend I have ordered...,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00,en,0.999996
335454,1542517994080702464,✦ Sakaja Question from an American,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00,en,0.999997
335455,1542518238566703108,Adams Arcade Ngong Road,1339887369495916545,-1.300567,36.780725,2022-06-30 14:39:41+00:00,en,0.714283
335456,1542521919118770184,You better recognize Owner kemeigabriel forest...,839394178824028160,-1.283330,36.816700,2022-06-30 14:54:18+00:00,da,0.714281


In [19]:
# Check all languages
UniqueLang = set(dfA.Language.tolist())
len(UniqueLang)

47

In [20]:
dfA['LangFreq'] = dfA.groupby('Language')['Language'].transform('count')
dfA.sort_values('LangFreq')

/var/folders/2_/nk9j6sb901n_fk5dz_9vtqj80000gn/T/ipykernel_31930/4148771128.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfA['LangFreq'] = dfA.groupby('Language')['Language'].transform('count')


,id,text,userid,lat,lon,created_at,Language,LangAcc,LangFreq
141070,1252823389196017664,ਦਮ ਹੋਣਾ ਚਾਹੀਦਾ ਜਨੂੰਨ ਨੂੰ ਜਿੰਦਾ ਰੱਖਣ ਲਈ ਤੇ ਸੋਚ ...,495707325,-1.280940,36.821678,2020-04-22 04:55:46+00:00,pa,1.000000,1
116404,1214865269656436738,ఆకాశం నీ హద్దురా Some Where Peaceful,2767195570,-1.300133,36.889654,2020-01-08 11:03:46+00:00,te,0.857141,1
180449,1318616895382237185,ಇಲ್ಲಿ ನೂರಾರು ಹಗಲಿಂದ ನಾಳಿನ ಇರುಳನ್ನು ತಡೆಯುವ ನರರಾ...,768512094270398465,-1.300133,36.889654,2020-10-20 18:15:41+00:00,kn,1.000000,1
271532,1451091183862751236,Публікує світлину,309580453,-1.283333,36.816667,2021-10-21 07:41:12+00:00,uk,0.999999,1
245239,1410857967944179719,જિંદગી એટલે જરૂરિયાત અને જવાબદારીઓ વચ્ચે પોતાન...,1407770601981452297,-1.300444,36.847260,2021-07-02 07:08:46+00:00,gu,1.000000,2
...,...,...,...,...,...,...,...,...,...
119294,1218854434303811584,The brunch at Nook Cafe is something else nook...,74134255,-1.294707,36.798934,2020-01-19 11:15:17+00:00,en,0.999998,251150
119293,1218854177490771968,When I was a Child I Walked like a Child I tal...,38396459,-1.283333,36.816667,2020-01-19 11:14:15+00:00,en,0.999997,251150
119292,1218852407364308992,What We SEE Depends Mainly On What We Look For...,504586023,-1.276389,36.814369,2020-01-19 11:07:13+00:00,en,0.999996,251150
119298,1218861104035782656,Areal view of Guji South Oromia region Courtes...,258745100,-1.276389,36.814369,2020-01-19 11:41:47+00:00,en,0.999997,251150


## Select only if language used more than 50 times

In [21]:
dfA2 = dfA.loc[dfA['LangFreq'] >= 50]
dfA2.sort_values('LangFreq')

,id,text,userid,lat,lon,created_at,Language,LangAcc,LangFreq
148048,1265242964483137537,arnoldsaviour 🤣🤣🤣🤣🤣 sammiekioko tomdaktari BM ...,1353606703,-1.284284,36.735299,2020-05-26 11:26:44+00:00,lv,0.714285,91
200442,1352570983568334849,Kibera Slums Niarobi,2177769462,-1.314920,36.778648,2021-01-22 10:57:06+00:00,lv,0.571427,91
104932,1200492417402753024,maumauzdaughter,212503069,-1.261499,36.912462,2019-11-29 19:11:11+00:00,lv,0.999995,91
248841,1416343700523954178,Our albums,56342678,-1.283333,36.816667,2021-07-17 10:27:07+00:00,lv,0.714284,91
27736,1111625483999956993,Damn funniestmemes funniestseconds funniestvid...,492151718,-1.283330,36.816700,2019-03-29 13:45:43+00:00,lv,0.714283,91
...,...,...,...,...,...,...,...,...,...
116340,1214802381122211840,January is all about goals⁣ And health should ...,276007575,-1.293487,36.762722,2020-01-08 06:53:52+00:00,en,0.999998,251150
116339,1214801460996853760,Burning the holiday calories and hunting for d...,246715678,-1.284210,36.869900,2020-01-08 06:50:12+00:00,en,0.999996,251150
116338,1214801422631546880,Hood Somewhere In Africa,725065969,-1.276389,36.814369,2020-01-08 06:50:03+00:00,en,0.999995,251150
116345,1214810407111577600,My WCWs this year so far lourinemaingi rehemam...,37870969,-1.280663,36.790028,2020-01-08 07:25:45+00:00,en,0.999997,251150


In [22]:
# Export valid frequency (language used more then 50 times) result
dfA2.to_csv(Export + 'LanGrt50.csv', index = False)

# Categorize into 3 language classes

In [23]:
# Check all languages2
UniqueLang2 = set(dfA2.Language.tolist())
# len(UniqueLang2)
UniqueLang2

{'af',
 'ar',
 'ca',
 'cy',
 'da',
 'de',
 'en',
 'es',
 'et',
 'fi',
 'fr',
 'hr',
 'hu',
 'id',
 'it',
 'ja',
 'lt',
 'lv',
 'nl',
 'no',
 'pl',
 'pt',
 'ro',
 'sk',
 'sl',
 'so',
 'sq',
 'sv',
 'sw',
 'tl',
 'tr',
 'vi'}

UniqueLang2af:Afrikaans, ar:Arabic, ca:Catalan, cs:Czech, cy:Welsh, da:Danish, de:German, es:Spanish, st:Estonian, fa:Persian, fi:Finnish, fr:French, hr:Croatian, hu':'Hungarian, id':'Indonesian, 'it':'Italian', ja':'Japanese', ko':'Korean, lt':'Lithuanian, 'lv':'Latvian', 'nl':'Dutch, no':'Norwegian, 'pl':'Polish', pt':'Portuguese', ro':'Romanian;, ru':'Russian, 'sk':'Slovak, sl':'Slovenian, 'so':'Somali, sq':'Albanian', sv':'Swedish, tl:Tagalog, tr':'Turkish',  vi':'Vietnamese, zh':'Chinese'

LOCAL: en:English, sw:Swahili,   
NON-LOCAL: others: everthing else more than 50 times 

In [28]:
# Categorize into 3 classes

dfsub1 = dfA2[dfA2['Language']=='en'] # Select English Tweets
dfsub1['LanguageCorr'] = 'en'
dfsub2 = dfA2[dfA2['Language']=='sw'] # Select SW Tweets
dfsub2['LanguageCorr'] = 'sw'
dfsub2

/var/folders/2_/nk9j6sb901n_fk5dz_9vtqj80000gn/T/ipykernel_31930/2152398069.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsub1['LanguageCorr'] = 'en'
/var/folders/2_/nk9j6sb901n_fk5dz_9vtqj80000gn/T/ipykernel_31930/2152398069.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsub2['LanguageCorr'] = 'sw'


,id,text,userid,lat,lon,created_at,Language,LangAcc,LangFreq,LanguageCorr
21,1079776044209659904,B L U E Waiyaki Waiyaki,613429043,-1.261522,36.802030,2018-12-31 16:27:24+00:00,sw,0.999996,12119,sw
26,1079780240552022022,kila mtu na mzigo wake,41765685,-1.283333,36.816667,2018-12-31 16:44:05+00:00,sw,0.999998,12119,sw
70,1079822272637136898,Ngoma ya kufungia mwaka hii moyokisoda kutoka ...,2502330008,-1.327230,36.852444,2018-12-31 19:31:06+00:00,sw,0.999997,12119,sw
151,1079888866427973632,Wajua mahali pa vyakula mi huwa chonjo sanaa 😂,127479463,-1.283330,36.816700,2018-12-31 23:55:43+00:00,sw,0.999996,12119,sw
286,1080080743198588929,Unakumbuka hizi brus pale mtaani time kanjo il...,118134064,-1.283330,36.816700,2019-01-01 12:38:10+00:00,sw,0.999995,12119,sw
...,...,...,...,...,...,...,...,...,...,...
335261,1542290687240044545,MM leo nimetoka nikatembea hadi most of my fee...,1269598733878079488,-1.292066,36.821945,2022-06-29 23:35:28+00:00,sw,0.999995,12119,sw
335311,1542382330668630018,Gishuvski Why,207486330,-1.280000,36.740000,2022-06-30 05:39:38+00:00,sw,0.999995,12119,sw
335433,1542492785256849409,Wapandishe bei ya kila kitu lakini wasiguse ki...,3437610928,-1.330607,36.860695,2022-06-30 12:58:32+00:00,sw,0.999998,12119,sw
335435,1542497223644176384,Kilimani,1359535373416886279,-1.289099,36.798138,2022-06-30 13:16:10+00:00,sw,0.999996,12119,sw


In [29]:
# Select non en/sw tweets
dfsub3 = dfA2[dfA2['Language']!='sw'] 
dfsub4 = dfsub3[dfsub3['Language']!='en']
dfsub4['LanguageCorr'] = 'Others'
dfsub4

/var/folders/2_/nk9j6sb901n_fk5dz_9vtqj80000gn/T/ipykernel_31930/638891507.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfsub4['LanguageCorr'] = 'Others'


,id,text,userid,lat,lon,created_at,Language,LangAcc,LangFreq,LanguageCorr
4,1079764407675015170,pm sunset,111780330,-1.270000,36.800000,2018-12-31 15:41:10+00:00,no,0.857138,1331,Others
16,1079771583898624000,Seed count 👌 zeeromoments,4344649527,-1.283330,36.816700,2018-12-31 16:09:41+00:00,nl,0.571428,1874,Others
23,1079778076563787777,Ok am soo ready for ya 😊,4523280328,-1.283333,36.816667,2018-12-31 16:35:29+00:00,so,0.571428,1555,Others
30,1079786933994377217,these three rats can make you go maaaaad famil...,263488167,-1.283520,36.823800,2018-12-31 17:10:41+00:00,so,0.999997,1555,Others
36,1079789446734770177,Wasafi Fest Uhuru Gardens in,288288362,-1.324884,36.797563,2018-12-31 17:20:40+00:00,de,0.999997,5395,Others
...,...,...,...,...,...,...,...,...,...,...
335437,1542500677301469184,Adams Arcade Ngong Road,1339887369495916545,-1.300567,36.780725,2022-06-30 13:29:54+00:00,pt,0.571426,1214,Others
335446,1542508718817849346,Car Connect,1010067177662238720,-1.267616,36.813812,2022-06-30 14:01:51+00:00,fr,0.999406,1980,Others
335447,1542510486142328834,Industrial Area Lungalunga Road,1142003483354324992,-1.306477,36.874567,2022-06-30 14:08:53+00:00,id,0.714284,5187,Others
335456,1542521919118770184,You better recognize Owner kemeigabriel forest...,839394178824028160,-1.283330,36.816700,2022-06-30 14:54:18+00:00,da,0.714281,1229,Others


In [30]:
# Put all together
dfA3 = pd.concat([dfsub1, dfsub2, dfsub4])
dfA3

,id,text,userid,lat,lon,created_at,Language,LangAcc,LangFreq,LanguageCorr
0,1079755309281890305,Quick Pose Before The Vibe KIZA KIZA Restauran...,149813541,-1.290832,36.782992,2018-12-31 15:05:01+00:00,en,0.999995,251150,en
1,1079759392138907649,whereveryouare haveagoodone beThankful Godsfa...,208112568,-1.283333,36.816667,2018-12-31 15:21:14+00:00,en,0.999996,251150,en
2,1079760884526972928,Happy from us,51949523,-1.283330,36.816700,2018-12-31 15:27:10+00:00,en,0.999998,251150,en
3,1079762365644660736,We ride together We shine together,89940972,-1.283330,36.816700,2018-12-31 15:33:03+00:00,en,0.999997,251150,en
5,1079765001454477312,We set to usher in the new year in style villa...,272970471,-1.269244,36.806866,2018-12-31 15:43:32+00:00,en,0.999997,251150,en
...,...,...,...,...,...,...,...,...,...,...
335437,1542500677301469184,Adams Arcade Ngong Road,1339887369495916545,-1.300567,36.780725,2022-06-30 13:29:54+00:00,pt,0.571426,1214,Others
335446,1542508718817849346,Car Connect,1010067177662238720,-1.267616,36.813812,2022-06-30 14:01:51+00:00,fr,0.999406,1980,Others
335447,1542510486142328834,Industrial Area Lungalunga Road,1142003483354324992,-1.306477,36.874567,2022-06-30 14:08:53+00:00,id,0.714284,5187,Others
335456,1542521919118770184,You better recognize Owner kemeigabriel forest...,839394178824028160,-1.283330,36.816700,2022-06-30 14:54:18+00:00,da,0.714281,1229,Others


In [31]:
# Export File with all columns
dfA3.to_csv(Export + 'Lan3class.csv', index = False)

In [ ]:
dfA3 = dfA3.drop(columns=['id','text', 'userid', 'created_at', 'location', 'HourOfTimestamp', 'Language', 'LangAcc', 'LangFreq'])
dfA3

# Match to Grid

### DataFrame to GeoDataFrame

In [36]:
# DataFrame to GeoDataFrame
gdfTwitter = gpd.GeoDataFrame(dfA3, geometry = [Point(xy) for xy in zip(dfA3.lon, dfA3.lat)])

# Add CRS
gdfTwitter.crs = 'epsg:4326'

# type(gdfTwitter)
# gdfTwitter

### Fishnet Shp file

In [33]:
shapefilePoly = gpd.read_file('/Users/wenlanzhang/Downloads/PhD_UCL/Data/Shp/NariobiFishnet/Polygon30/NariobiFishnet.shp')

# Drop unwanted columns
shapefilePoly = shapefilePoly.drop(columns=['Shape_Area','Grid','AREA','OBJECTID','PERIMETER',
                                            'COUNTY3_', 'COUNTY3_ID','COUNTY', 'Shape_Leng'])

# Add A Column
# shapefilePoly['index'] = shapefilePoly.index   # Will generate automatically

shapefilePoly['GeometryBUP'] = shapefilePoly['geometry']

# type(shapefilePoly)
shapefilePoly

,geometry,GeometryBUP
0,"POLYGON ((36.89920 -1.16017, 36.89902 -1.16017...","POLYGON ((36.89920 -1.16017, 36.89902 -1.16017..."
1,"POLYGON ((36.89920 -1.16016, 36.89947 -1.16015...","POLYGON ((36.89920 -1.16016, 36.89947 -1.16015..."
2,"POLYGON ((36.89947 -1.16015, 36.89974 -1.16014...","POLYGON ((36.89947 -1.16015, 36.89974 -1.16014..."
3,"POLYGON ((36.89974 -1.16014, 36.90001 -1.16012...","POLYGON ((36.89974 -1.16014, 36.90001 -1.16012..."
4,"POLYGON ((36.90001 -1.16012, 36.90028 -1.16011...","POLYGON ((36.90001 -1.16012, 36.90028 -1.16011..."
...,...,...
792529,"POLYGON ((36.95903 -1.44341, 36.95924 -1.44341...","POLYGON ((36.95903 -1.44341, 36.95924 -1.44341..."
792530,"POLYGON ((36.95876 -1.44368, 36.95876 -1.44388...","POLYGON ((36.95876 -1.44368, 36.95876 -1.44388..."
792531,"POLYGON ((36.95876 -1.44368, 36.95903 -1.44368...","POLYGON ((36.95876 -1.44368, 36.95903 -1.44368..."
792532,"POLYGON ((36.95903 -1.44368, 36.95912 -1.44368...","POLYGON ((36.95903 -1.44368, 36.95912 -1.44368..."


### Merge/Intersect Data

In [37]:
# Intersection & Clean-up
from tqdm import tqdm

for i in tqdm(range(20), desc = 'tqdm() Progress Bar'):
    MatchedTwitter = gpd.sjoin(left_df = gdfTwitter, 
                               right_df = shapefilePoly[['geometry','GeometryBUP']], 
                               op='intersects')

MatchedTwitter
# Time: 10min

tqdm() Progress Bar: 100%|██████████████████████| 20/20 [08:34<00:00, 25.71s/it]


,id,text,userid,lat,lon,created_at,Language,LangAcc,LangFreq,LanguageCorr,geometry,index_right,GeometryBUP
0,1079755309281890305,Quick Pose Before The Vibe KIZA KIZA Restauran...,149813541,-1.290832,36.782992,2018-12-31 15:05:01+00:00,en,0.999995,251150,en,POINT (36.78299 -1.29083),415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
88,1079845360330293248,Smiling all the way into like😁😁 To lovelife an...,254550008,-1.290832,36.782992,2018-12-31 21:02:51+00:00,en,0.999995,251150,en,POINT (36.78299 -1.29083),415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
174,1079946787870392320,Swizz footballers were in the building One lov...,1459793124,-1.290773,36.782847,2019-01-01 03:45:53+00:00,en,0.999998,251150,en,POINT (36.78285 -1.29077),415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
189,1079975641968070656,Dance Party Memories created BCLUB Entertainme...,1459793124,-1.290773,36.782847,2019-01-01 05:40:32+00:00,en,0.999997,251150,en,POINT (36.78285 -1.29077),415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
1018,1080937020355096577,Your view not mine BCLUB ArmaniTheHitmaker BClub,1459793124,-1.290773,36.782847,2019-01-03 21:20:43+00:00,en,0.999994,251150,en,POINT (36.78285 -1.29077),415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
333491,1539928633405591553,Runda,839508948,-1.290219,36.772360,2022-06-23 11:09:31+00:00,tr,0.857141,713,Others,POINT (36.77236 -1.29022),412207,"POLYGON ((36.77227 -1.29007, 36.77254 -1.29007..."
333796,1540308166931628035,The tea 🍵 Muranga Gaturi,773049704854814720,-1.219956,36.891068,2022-06-24 12:17:39+00:00,id,0.571428,5187,Others,POINT (36.89107 -1.21996),61154,"POLYGON ((36.89084 -1.21973, 36.89111 -1.21973..."
333818,1540336699917475842,Mariakani Gardens South B,1141292695786328064,-1.309739,36.835095,2022-06-24 14:11:01+00:00,id,0.714285,5187,Others,POINT (36.83509 -1.30974),500541,"POLYGON ((36.83506 -1.30947, 36.83533 -1.30947..."
334237,1541007289833988097,Im at SMA Stadium in,77694324,-1.340698,36.757501,2022-06-26 10:35:43+00:00,de,0.571427,5395,Others,POINT (36.75750 -1.34070),623754,"POLYGON ((36.75744 -1.34047, 36.75771 -1.34047..."


In [49]:
dfLan = MatchedTwitter[['id','LanguageCorr','index_right', 'GeometryBUP']]
dfLan

,id,LanguageCorr,index_right,GeometryBUP
0,1079755309281890305,en,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
88,1079845360330293248,en,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
174,1079946787870392320,en,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
189,1079975641968070656,en,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
1018,1080937020355096577,en,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
...,...,...,...,...
333491,1539928633405591553,Others,412207,"POLYGON ((36.77227 -1.29007, 36.77254 -1.29007..."
333796,1540308166931628035,Others,61154,"POLYGON ((36.89084 -1.21973, 36.89111 -1.21973..."
333818,1540336699917475842,Others,500541,"POLYGON ((36.83506 -1.30947, 36.83533 -1.30947..."
334237,1541007289833988097,Others,623754,"POLYGON ((36.75744 -1.34047, 36.75771 -1.34047..."


### Export

In [50]:
dfLan.to_csv(Export + 'Grided3LanClass.csv', index = False)

# Get Dummies

In [51]:
# df_dc = pd.get_dummies(dfA2, columns=['Language'])
df_dc = pd.get_dummies(dfLan, columns=['LanguageCorr'])
df_dc

,id,index_right,GeometryBUP,LanguageCorr_Others,LanguageCorr_en,LanguageCorr_sw
0,1079755309281890305,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",0,1,0
88,1079845360330293248,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",0,1,0
174,1079946787870392320,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",0,1,0
189,1079975641968070656,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",0,1,0
1018,1080937020355096577,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",0,1,0
...,...,...,...,...,...,...
333491,1539928633405591553,412207,"POLYGON ((36.77227 -1.29007, 36.77254 -1.29007...",1,0,0
333796,1540308166931628035,61154,"POLYGON ((36.89084 -1.21973, 36.89111 -1.21973...",1,0,0
333818,1540336699917475842,500541,"POLYGON ((36.83506 -1.30947, 36.83533 -1.30947...",1,0,0
334237,1541007289833988097,623754,"POLYGON ((36.75744 -1.34047, 36.75771 -1.34047...",1,0,0


In [52]:
df_dc.rename(columns={'LanguageCorr_Others': 'Others', 
                      'LanguageCorr_en': 'En', 
                      'LanguageCorr_sw': 'Sw'}, inplace=True)
df_dc = df_dc.astype({"Others": int, "En": int, 'Sw':int })
df_dc

,id,index_right,GeometryBUP,Others,En,Sw
0,1079755309281890305,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",0,1,0
88,1079845360330293248,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",0,1,0
174,1079946787870392320,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",0,1,0
189,1079975641968070656,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",0,1,0
1018,1080937020355096577,415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",0,1,0
...,...,...,...,...,...,...
333491,1539928633405591553,412207,"POLYGON ((36.77227 -1.29007, 36.77254 -1.29007...",1,0,0
333796,1540308166931628035,61154,"POLYGON ((36.89084 -1.21973, 36.89111 -1.21973...",1,0,0
333818,1540336699917475842,500541,"POLYGON ((36.83506 -1.30947, 36.83533 -1.30947...",1,0,0
334237,1541007289833988097,623754,"POLYGON ((36.75744 -1.34047, 36.75771 -1.34047...",1,0,0


# Combine by pixcel

In [53]:
# Select repeated pixcel

Uid = list(set(df_dc.index_right.tolist()))
# print(Uid)
print(len(Uid))

12119


In [54]:
df_dc.columns

Index(['id', 'index_right', 'GeometryBUP', 'Others', 'En', 'Sw'], dtype='object')

In [68]:
# Combine language by Pixcel 

i = 0
dfnew = pd.DataFrame(columns = df_dc.columns)

start_time = time.monotonic()
for id in Uid:
# id = 524290
    dftemp = df_dc[df_dc['index_right'] == id]
    dftemp = dftemp.reset_index()
    dfnew.at[i,'id'] = dftemp.loc[0]['id']
    dfnew.at[i,'GeometryBUP'] = dftemp.loc[0]['GeometryBUP']
    dfnew.at[i,'index_right'] = dftemp.loc[0]['index_right']
    dfnew.at[i,'Others'] = dftemp['Others'].sum()  
    dfnew.at[i,'En'] = dftemp['En'].sum()  
    dfnew.at[i,'Sw'] = dftemp['Sw'].sum()  

    i = i + 1

end_time = time.monotonic()
print(timedelta(seconds = end_time - start_time))

# type(dfnew)
dfnew

0:00:12.403188


,id,index_right,GeometryBUP,Others,En,Sw
0,1205558302668722177,327684,"POLYGON ((36.8099959027786 -1.274439893580406,...",0,10,1
1,1142160853296406534,524293,POLYGON ((36.89865962132013 -1.314864081365799...,1,3,0
2,1175046619101892609,425988,POLYGON ((36.77577009045405 -1.292765525376451...,1,0,0
3,1179109342907252737,524296,POLYGON ((36.89946810507583 -1.314864081365799...,1,1,0
4,1266807841677074432,393224,POLYGON ((36.90108507258722 -1.286297655330788...,1,2,0
...,...,...,...,...,...,...
12114,1096788749386768385,196599,POLYGON ((36.72780005428265 -1.250724370079642...,1,0,0
12115,1243222379867840513,524280,"POLYGON ((36.8951561917121 -1.314864081365799,...",1,0,0
12116,1346409324420149248,425981,POLYGON ((36.77388362835742 -1.292765525376451...,0,1,0
12117,1090155886109212672,622590,POLYGON ((36.69249626361717 -1.340196572377978...,0,1,0


## Export to csv

In [69]:
dfnew.to_csv(Export + '3ClassDummy.csv', index = False)

## Export to shp

In [70]:
# DataFrame to GeoDataFrame
gdfdc = gpd.GeoDataFrame(dfnew, crs="epsg:4326", geometry = dfnew['GeometryBUP'])

# type(gdfdc)
gdfdc

,id,index_right,GeometryBUP,Others,En,Sw,geometry
0,1205558302668722177,327684,"POLYGON ((36.8099959027786 -1.274439893580406,...",0,10,1,"POLYGON ((36.81000 -1.27444, 36.81027 -1.27444..."
1,1142160853296406534,524293,POLYGON ((36.89865962132013 -1.314864081365799...,1,3,0,"POLYGON ((36.89866 -1.31486, 36.89893 -1.31486..."
2,1175046619101892609,425988,POLYGON ((36.77577009045405 -1.292765525376451...,1,0,0,"POLYGON ((36.77577 -1.29277, 36.77604 -1.29277..."
3,1179109342907252737,524296,POLYGON ((36.89946810507583 -1.314864081365799...,1,1,0,"POLYGON ((36.89947 -1.31486, 36.89974 -1.31486..."
4,1266807841677074432,393224,POLYGON ((36.90108507258722 -1.286297655330788...,1,2,0,"POLYGON ((36.90109 -1.28630, 36.90135 -1.28630..."
...,...,...,...,...,...,...,...
12114,1096788749386768385,196599,POLYGON ((36.72780005428265 -1.250724370079642...,1,0,0,"POLYGON ((36.72780 -1.25072, 36.72807 -1.25072..."
12115,1243222379867840513,524280,"POLYGON ((36.8951561917121 -1.314864081365799,...",1,0,0,"POLYGON ((36.89516 -1.31486, 36.89543 -1.31486..."
12116,1346409324420149248,425981,POLYGON ((36.77388362835742 -1.292765525376451...,0,1,0,"POLYGON ((36.77388 -1.29277, 36.77415 -1.29277..."
12117,1090155886109212672,622590,POLYGON ((36.69249626361717 -1.340196572377978...,0,1,0,"POLYGON ((36.69250 -1.34020, 36.69277 -1.34020..."


In [ ]:
gdfdc = gdfdc.drop(columns=['GeometryBUP', 'index_right'])
# type(gdfdc)
gdfdc

In [82]:
gdfdc['Others'] = gdfdc['Others'].astype("int")
gdfdc['En'] = gdfdc['En'].astype("int")
gdfdc['Sw'] = gdfdc['Sw'].astype("int")
gdfdc['id'] = gdfdc['id'].astype("int")

gdfdc.dtypes

id             int64
Others         int64
En             int64
Sw             int64
geometry    geometry
dtype: object

In [84]:
gdfdc

,id,Others,En,Sw,geometry
0,1205558302668722177,0,10,1,"POLYGON ((36.81000 -1.27444, 36.81027 -1.27444..."
1,1142160853296406534,1,3,0,"POLYGON ((36.89866 -1.31486, 36.89893 -1.31486..."
2,1175046619101892609,1,0,0,"POLYGON ((36.77577 -1.29277, 36.77604 -1.29277..."
3,1179109342907252737,1,1,0,"POLYGON ((36.89947 -1.31486, 36.89974 -1.31486..."
4,1266807841677074432,1,2,0,"POLYGON ((36.90109 -1.28630, 36.90135 -1.28630..."
...,...,...,...,...,...
12114,1096788749386768385,1,0,0,"POLYGON ((36.72780 -1.25072, 36.72807 -1.25072..."
12115,1243222379867840513,1,0,0,"POLYGON ((36.89516 -1.31486, 36.89543 -1.31486..."
12116,1346409324420149248,0,1,0,"POLYGON ((36.77388 -1.29277, 36.77415 -1.29277..."
12117,1090155886109212672,0,1,0,"POLYGON ((36.69250 -1.34020, 36.69277 -1.34020..."


In [85]:
gdfdc.to_file(Export + 'Grided.shp', index = False) 